In [1]:
import numpy as np


In [2]:
from typing import List


class Solution:
    def minCost(self, nums: List[int], cost: List[int]) -> int:
        def get_cost(base):
            return sum(abs(base - num) * c for num, c in zip(nums, cost))

        # Initialize the left and the right boundary of the binary search.
        left, right = min(nums), max(nums)
        answer = get_cost(nums[0])

        # As shown in the previous picture, if F(mid) > F(mid + 1), then the minimum
        # is to the right of mid, otherwise, the minimum is to the left of mid.
        while left < right:
            mid = (left + right) // 2
            cost_1 = get_cost(mid)
            cost_2 = get_cost(mid + 1)
            answer = min(cost_1, cost_2)

            if cost_1 > cost_2:
                left = mid + 1
            else:
                right = mid

        return answer


## Dãy con lập thành cấp số cộng dài nhất


In [3]:
class Solution:
    def longestArithSeqLength(self, nums: List[int]) -> int:
        dp = {}
        for r in range(1, len(nums)):
            for l in range(r):
                diff = nums[r] - nums[l]
                dp[r, diff] = dp.get((l, diff), 1) + 1
        return max(dp.values())


%time Solution().longestArithSeqLength(np.random.randint(0, 500, 1000))


CPU times: total: 438 ms
Wall time: 482 ms


10

In [4]:
from time import time
from collections import defaultdict

np.random.seed(0)
# nums = list(np.random.randint(0, 50, 20))
nums = [24, 33, 23, 23, 29, 24, 31, 29, 36,
        32, 22, 18, 35, 36, 37, 41, 23, 39, 33, 32]
n = len(nums)
s = [0] * (1 << n)
candidates = []
mark = defaultdict(list)
s_num = sum(nums)
start = time()
for i in range(1, 1 << n):
    for j in range(n):
        if i & (1 << j):
            s[i] = s[i ^ (1 << j)] + nums[j]
            if s[i] <= s_num // 2:
                candidates.append(s[i])
            mark[s[i]].append(i)
            break
candidates = sorted(set(candidates), reverse=True)
mark[0] = [0]
ans = 0
for num in candidates:
    if ans != 0:
        break
    for i in mark[s_num - 2*num]:
        for j in mark[num]:
            if i & j == 0:
                ans = num
                break
        if ans != 0:
            break
print(time() - start)
print(ans)


1.577779769897461
300


In [5]:
from functools import lru_cache


class Solution:
    def countRoutes(self, locations: List[int], start: int, finish: int, fuel: int) -> int:
        n = len(locations)
        mod = 10**9 + 7

        @lru_cache(maxsize=None)
        def dfs(i, fuel):
            count = 0
            for j in range(n):
                if i != j:
                    x = fuel - abs(locations[i] - locations[j])
                    if x >= 0:
                        count = (count + dfs(j, x)) % mod
                        if j == finish:
                            count = (count + 1) % mod
            return count
        if start == finish:
            return (dfs(start, fuel) + 1) % mod
        return dfs(start, fuel)


In [6]:
from heapq import heapify, heappop, heappush


class Solution:
    def totalCost(self, costs: List[int], k: int, candidates: int) -> int:
        pq = []
        for i in range(candidates):
            pq.append((costs[i], 0))
        for i in range(max(candidates, len(costs) - candidates), len(costs)):
            pq.append((costs[i], 1))

        heapify(pq)

        answer = 0
        next_head, next_tail = candidates, len(costs) - 1 - candidates

        for _ in range(k):
            cur_cost, cur_section_id = heappop(pq)
            answer += cur_cost
            if next_head <= next_tail:
                if cur_section_id == 0:
                    heappush(pq, (costs[next_head], 0))
                    next_head += 1
                else:
                    heappush(pq, (costs[next_tail], 1))
                    next_tail -= 1

        return answer


In [7]:
class Solution:
    def maxProbability(self, n: int, edges: List[List[int]], succProb: List[float], start: int, end: int) -> float:
        adj = defaultdict(list)
        for i in range(len(edges)):
            adj[edges[i][0]].append((edges[i][1], succProb[i]))
            adj[edges[i][1]].append((edges[i][0], succProb[i]))
        visited = [False] * n
        visited[start] = True
        prob = [0] * n
        prob[start] = 1
        pq = [(0, start)]
        while pq:
            _, node = heappop(pq)
            if visited[node]:
                continue
            visited[node] = True
            for neighbor, p in adj[node]:
                if prob[node] * p > prob[neighbor]:
                    prob[neighbor] = prob[node] * p
                    heappush(pq, (prob[neighbor], neighbor))
        return prob[end]


In [10]:
class Solution:
    def kthFactor(self, n: int, k: int) -> int:
        factors = []
        for i in range(1, int(n**0.5) + 1):
            if n % i == 0:
                factors.append(i)
        x = len(factors)
        if k>2*x:
            return -1
        if k<=x:
            return factors[k-1]
        return n//factors[x-k]
s = Solution()
s.kthFactor(100)

-1

In [ ]:
from collections import defaultdict, deque

class Solution:
    def shortestPathAllKeys(self, grid: List[str]) -> int:
        rows, cols = len(grid), len(grid[0])
        queue = deque()
        visited = defaultdict(set)
        keys_set, locks_set = set(), set()
        all_keys = 0
        start_row, start_col = -1, -1
        
        for i in range(rows):
            for j in range(cols):
                cell = grid[i][j]
                if cell in 'abcdef':
                    all_keys += (1 << (ord(cell) - ord('a')))
                    keys_set.add(cell)
                if cell in 'ABCDEF':
                    locks_set.add(cell)
                if cell == "@":
                    start_row, start_col = i, j
        
        queue.append((start_row, start_col, 0, 0))
        visited[0].add((start_row, start_col))
        
        while queue:
            cur_row, cur_col, keys, dist = queue.popleft()
            for dr, dc in ((0, 1), (1, 0), (-1, 0), (0, -1)):
                new_row, new_col = cur_row + dr, cur_col + dc
                
                if 0 <= new_row < rows and 0 <= new_col < cols and grid[new_row][new_col] != '#':
                    cell = grid[new_row][new_col]
                    
                    if cell in keys_set and not ((1 << (ord(cell) - ord('a'))) & keys):
                        new_keys = (keys | (1 << (ord(cell) - ord('a'))))
                        if new_keys == all_keys:
                            return dist + 1
                        visited[new_keys].add((new_row, new_col))
                        queue.append((new_row, new_col, new_keys, dist + 1))
                      
                    elif cell in locks_set and not (keys & (1 << (ord(cell) - ord('A')))):
                        continue
                        
                    elif (new_row, new_col) not in visited[keys]:
                        visited[keys].add((new_row, new_col))
                        queue.append((new_row, new_col, keys, dist + 1))
            
        return -1


In [ ]:
# Define a class for Disjoint Set Union (DSU)
class DSU:
    # Initialize the DSU with a list of roots and a list of sizes
    def __init__(self, n):
        self.root = list(range(n))
        self.size = [1] * n

    # Find the root of a given element
    def find(self, x):
        if self.root[x] != x:
            self.root[x] = self.find(self.root[x])
        return self.root[x]

    # Union two elements
    def union(self, x, y):
        # Find the roots of the two elements
        root_x = self.find(x)
        root_y = self.find(y)
        # If the roots are the same, the elements are already in the same set
        if root_x == root_y:
            return
        # Otherwise, merge the smaller set into the larger set
        if self.size[root_x] > self.size[root_y]:
            root_x, root_y = root_y, root_x
        self.root[root_x] = root_y
        self.size[root_y] = self.size[root_x]

# Define a class for the solution
class Solution:
    # Define a function to find the latest day to cross
    def latestDayToCross(self, row: int, col: int, cells: List[List[int]]) -> int:
        # Initialize a DSU with row * col + 2 elements
        dsu = DSU(row * col + 2)
        # Initialize a grid with zeros
        grid = [[0] * col for _ in range(row)]
        # Define the eight possible directions to move in
        directions = [(0, 1), (0, -1), (1, 0), (-1, 0), (1, 1), (1, -1), (-1, 1), (-1, -1)]

        # Iterate over each cell
        for i in range(row * col):
            # Get the row and column of the cell
            r, c = cells[i][0] - 1, cells[i][1] - 1
            # Mark the cell as occupied
            grid[r][c] = 1
            # Get the index of the cell in the DSU
            index_1 = r * col + c + 1
            # Iterate over each direction
            for dr, dc in directions:
                # Get the row and column of the neighboring cell
                new_r, new_c = r + dr, c + dc
                # Get the index of the neighboring cell in the DSU
                index_2 = new_r * col + new_c + 1
                # If the neighboring cell is occupied, union the two cells in the DSU
                if 0 <= new_r < row and 0 <= new_c < col and grid[new_r][new_c] == 1:
                    dsu.union(index_1, index_2)
                    
            # If the cell is on the left edge, union it with the "start" element in the DSU
            if c == 0:
                dsu.union(0, index_1)
            # If the cell is on the right edge, union it with the "end" element in the DSU
            if c == col - 1:
                dsu.union(row * col + 1, index_1)
            # If the "start" and "end" elements are in the same set, we can cross the river
            if dsu.find(0) == dsu.find(row * col + 1):
                return i